In [1]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from typing import (Iterable, Callable, Mapping, TypeVar, 
                    List, Tuple, Optional,Sequence, Iterator)
from rl.markov_process import MarkovRewardProcess
from rl.markov_decision_process import Policy,MarkovDecisionProcess,FinitePolicy
from rl.function_approx import Tabular, FunctionApprox
from rl.dynamic_programming import policy_iteration_result,value_iteration_result
from rl.iterate import last,iterate
from rl.approximate_dynamic_programming import evaluate_mrp, value_iteration

from rl.chapter3.simple_inventory_mdp_cap import SimpleInventoryMDPCap, InventoryState
from rl.chapter7.asset_alloc_discrete import AssetAllocDiscrete 
from rl.monte_carlo import mc_prediction, mc_control
from rl.td import td_prediction

from rl.distribution import Constant,Distribution,Choose

In [ ]:
S = TypeVar('S')
A = TypeVar('S')

def policy_iteration(
    mdp: MarkovDecisionProcess[S, A],
    γ: float,
    approx_0: FunctionApprox[S],
    non_terminal: Distribution[S],
    num_state_samples: int
) -> Iterator[FunctionApprox[S]]:
    def update(v: Tuple[FunctionApprox[S], Policy[S, A]]
              ) -> Tuple[FunctionApprox[S], Policy[S, A]]:
        vf, pi = vf_policy
        mrp: MarkovRewardProcess[S] = mdp.apply_policy(pi)
        policy_vf: FunctionApprox[S] = last(evaluate_mrp(mrp, γ, vf, 
                                                         non_terminal, num_state_samples))    
        improved_pi: Policy[S, A] = greedy_policy_from_vf(mdp,policy_vf,γ)
        return policy_vf, improved_pi
    pi_0 = policy_from_v(approx_0,mdp)
    return iterate(update, (approx_0, pi_0))